In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import feature_builder

Primero cargamos los datasets

In [2]:
test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [3]:
def train_knn(x, y, k):
    count_vect = CountVectorizer()
    tfidf_transformer = TfidfTransformer()

    x_train_counts = count_vect.fit_transform(x)
    x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)
    
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train_tfidf, y)
    return (knn, count_vect, tfidf_transformer)

In [4]:
def predict_knn(knn_tuple, x_test, ids):
    knn, count_vect, tfidf_transformer = knn_tuple
    x_test_counts = count_vect.transform(x_test)
    x_test_tfidf = tfidf_transformer.transform(x_test_counts)
    targets = knn.predict(x_test_tfidf)
    return pd.DataFrame({'target':targets}, index=ids)

In [5]:
x = train_df['text']
y = train_df['target']

knn = train_knn(x, y, 5)
prediction_df = predict_knn(knn, test_df['text'], test_df['id'])

Sobre el texto del tweet con K = 5 esto nos dio una accuaracy de 0.77229

Ahora voy a hacer un ensamble de 3 modelos de KNNs uno de texto uno del location y uno del topic

In [6]:
train_df['location'] = train_df['location'].fillna('NAN_LOCATION')
train_df['keyword'] = train_df['keyword'].fillna('NAN_KEYWORD')

test_df['location'] = test_df['location'].fillna('NAN_LOCATION')
test_df['keyword'] = test_df['keyword'].fillna('NAN_KEYWORD')

In [7]:
k = 16
knn_location = train_knn(train_df['location'], train_df['target'], k)
knn_keyword = train_knn(train_df['keyword'], train_df['target'], k)
knn_text = train_knn(train_df['text'], train_df['target'], k)

location_prediction_df = predict_knn(knn_location, test_df['location'], test_df['id'])
keyword_prediction_df = predict_knn(knn_keyword, test_df['keyword'], test_df['id'])
text_prediction_df = predict_knn(knn_text, test_df['text'], test_df['id'])

final_df = pd.merge(pd.merge(text_prediction_df, keyword_prediction_df, on='id'), location_prediction_df, on='id')

In [8]:
final_df['target'] = (final_df['target_x'] + final_df['target_y'] + final_df['target']) / 3
final_df = final_df.drop(columns=['target_x', 'target_y'])
final_df['target'] = final_df['target'].apply(lambda x: 1.0 if x >= 0.5 else 0.0)
final_df = final_df.astype({'target': 'int32'})

In [9]:
final_df.to_csv('knn.csv')
final_df['target'].value_counts()

0    2359
1     904
Name: target, dtype: int64

Score del ensamble: 0.73827

In [1]:
xtrain_df, xvalid_df, ytrain, yvalid_df = train_test_split(train_df, train_df['target'].values, test_size=0.2,random_state=1)
x_train = feature_builder.process_dataset(xtrain_df)
x_valid = feature_builder.process_dataset(xvalid_df)
knn = KNeighborsClassifier(n_neighbors=8)
knn.fit(x_train, ytrain_df)
targets = knn.predict(x_valid)


NameError: name 'train_test_split' is not defined